<h1>Tests</h1>

In [1]:
import os
import tensorflow as tf
from scipy import signal
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#For a google drive use :
model = tf.keras.models.load_model('/content/drive/MyDrive/ENSC - Travail Mathys/2A/S8/PII/Code/screamDetection.h5')
#For a local use :
#localFolder = os.getcwd()
#model = tf.keras.models.load_model(os.path.join(localFile,'screamDetection.h5'))

In [3]:
#Data loading function
def load_wav_16k_mono(filename):
    # Load encoded wav file
    file_contents = tf.io.read_file(filename)
    # Decode wav (tensors by channels)
    wav, sample_rate = tf.audio.decode_wav(file_contents, desired_channels=1)
    return tf.squeeze(wav,axis=-1)

In [4]:
#Preprocessing function
def preprocess_samples (sample, index):
    sample = sample[0]
    zero_padding = tf.zeros([130000] - tf.shape(sample), dtype=tf.float32)
    wav = tf.concat([zero_padding,sample],0)
    spectrogram = tf.signal.stft(wav, frame_length=256, frame_step=32)
    spectrogram = tf.abs(spectrogram)
    spectrogram = tf.expand_dims(spectrogram, axis=2)
    return spectrogram

In [25]:
#Put the link of your file you want to test
test = '/content/drive/MyDrive/ENSC - Travail Mathys/2A/S8/PII/Code/data/test/0x.wav'

In [26]:
test = load_wav_16k_mono(test)

In [27]:
#Slice the audio in 130000-length pieces (average length of a scream) 
if len(test) > 130000:
  audio_slices = tf.keras.utils.timeseries_dataset_from_array(test, test, sequence_length=130000, sequence_stride=130000, batch_size=1)
else:
  audio_slices = tf.keras.utils.timeseries_dataset_from_array(test, test, sequence_length=len(test)-1, sequence_stride=len(test)-1, batch_size=1)

In [28]:
samples, index = audio_slices.as_numpy_iterator().next()

In [29]:
#Data treatment
audio_slices = audio_slices.map(preprocess_samples)
audio_slices = audio_slices.batch(64)

In [30]:
#Predicting how many screams in the audio
yhat = model.predict(audio_slices)
yhat = [1 if prediction > 0.99 else 0 for prediction in yhat]
#Group consecutive detections
from itertools import groupby
yhat = [key for key, group in groupby(yhat)]
#Count how many sceams are detected
tf.math.reduce_sum(yhat)

1/1 [==============================] - 0s 266ms/step


<tf.Tensor: shape=(), dtype=int32, numpy=0>